In [1]:
import pandas as pd
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.datasets import fetch_california_housing

from sklearn.preprocessing import StandardScaler

In [2]:


# Load the California Housing dataset
housing = fetch_california_housing(as_frame=True)

# Create a Pandas DataFrame from the dataset
data = pd.DataFrame(housing.data, columns=housing.feature_names)

# Add the target variable to the DataFrame
data['target'] = housing.target

# 1. Normalize each feature using StandardScaler
scaler = StandardScaler()
data[data.columns[:-1]] = scaler.fit_transform(data[data.columns[:-1]])

# 2. Identify and drop extreme values (outliers)
# You can define a threshold to determine what is considered an extreme value.
# For example, you can use a z-score to detect outliers beyond a certain threshold.

threshold = 3  # Adjust this threshold as needed

# Calculate z-scores for each feature
z_scores = (data[data.columns[:-1]] - data[data.columns[:-1]].mean()) / data[data.columns[:-1]].std()

# Create a boolean mask for extreme values
outlier_mask = (z_scores.abs() < threshold).all(axis=1)

# Filter the DataFrame to remove rows with extreme values
data = data[outlier_mask]
data = data.reset_index(drop=True)

# Now, data contains the normalized features, and extreme values have been removed.


In [3]:
import sys
sys.path.append('..')
from xDL.models.NAM import NAM

In [4]:
nam = NAM(
    "target ~  -1 + PolynomialSplineNet(Longitude; degree=8) + CubicSplineNet(AveRooms; n_knots=8)+ CubicSplineNet(MedInc; n_knots=8) + CubicSplineNet(Latitude; n_knots=8) ", 
    data=data, 
    feature_dropout=0.0001
    )

--- Preprocessing ---


5it [03:01, 36.26s/it]
4it [00:08,  2.05s/it]


In [5]:
nam.training_dataset

<_PrefetchDataset element_spec=({'Longitude': TensorSpec(shape=(None, 8), dtype=tf.float32, name=None), 'AveRooms': TensorSpec(shape=(None, 8), dtype=tf.float64, name=None), 'MedInc': TensorSpec(shape=(None, 8), dtype=tf.float64, name=None), 'Latitude': TensorSpec(shape=(None, 8), dtype=tf.float64, name=None)}, TensorSpec(shape=(None,), dtype=tf.float64, name=None))>

In [6]:
nam.compile(optimizer=Adam(learning_rate=0.01), loss={"output": "mse"})

# Train the model
nam.fit(nam.training_dataset, epochs=50, validation_data=nam.validation_dataset)

# Evaluate the model
loss = nam.evaluate(nam.validation_dataset)
print("Test Loss:", loss)

Epoch 1/50


------------- Network architecture --------------
Longitude -> PolynomialSplineNet(feature=Longitude, n_params=648) -> output dimension=1
AveRooms -> CubicSplineNet(feature=AveRooms, n_params=648) -> output dimension=1
MedInc -> CubicSplineNet(feature=MedInc, n_params=648) -> output dimension=1
Latitude -> CubicSplineNet(feature=Latitude, n_params=648) -> output dimension=1
16/16 [==============================] - 3s 24ms/step - loss: 26.6401 - output_loss: 26.6037 - val_loss: 10.3892 - val_output_loss: 10.3623
Epoch 2/50
16/16 [==============================] - 0s 11ms/step - loss: 3.0099 - output_loss: 2.9874 - val_loss: 2.5590 - val_output_loss: 2.5362
Epoch 3/50
16/16 [==============================] - 0s 11ms/step - loss: 1.4281 - output_loss: 1.4063 - val_loss: 1.2931 - val_output_loss: 1.2717
Epoch 4/50
16/16 [==============================] - 0s 10ms/step - loss: 0.8226 - output_loss: 0.8014 - val_loss: 0.9886 - val_output_loss: 0.9675
Epoch 5/50
16/16 [========================